# Eminem RNN

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Imports
import sys
import numpy as np
import string
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer, one_hot, hashing_trick
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator, pad_sequences
from keras.preprocessing.text import text_to_word_sequence

from random import randint
from pickle import load, dump

import io
import pandas as pd
import time
import re

In [7]:
lyrics_by_song_with_headers = pd.read_csv('/content/drive/My Drive/Eminem Lyrics with Headers', index_col=0)
lyrics_by_song_with_headers.head()

,"Any rapper saying those kind of rhymes In this day and age in this period of time Tryna battle Eminem is worse than David Starr tryna battle Proof, Eye-Ku, BFlat and Bizarre (unintelligible) That's why you beg to differ me, you ain't got no style Plus you lack delivery, not to brag I don't need to boast Look, my face is pale, but you look like you seen a ghost (unintelligible) You couldn't slip up in the zone, if I put banana peels Around this fucking microphone So get a bulldozer, start from bottom to top And tear this building down 'cause that's the only way you wrecking shop"
1996 Underground Freestyle,
"1997 Freestyle Live at Wetlands, NYC","[Verse 1] Everybody, duck down, all you hear i..."
1997 Rap Olympics,[RAP BATTLE 1] I'ma tell you this for your own...
1999,"[Intro] Hi kids, do you like Kim's bullshit? (..."
1999 Tim Westwood Freestyle,"[Verse 1: Eminem] Tim Westwood, Marley Marl an..."
2004 Tim Westwood Freestyle,[Verse 1: Stat Quo] I boss how I live Fuck 'em...


In [8]:
f = open('/content/drive/My Drive/lyrics_eminem.txt', 'r')

with open('/content/drive/My Drive/lyrics_eminem.txt', 'r') as f:
    text = f.read()
type(text) # 1 huge string

str

In [0]:
# Create function to clean and tokenize text
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

In [0]:
# Create function to save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

In [11]:
# clean my text of lyrics
tokens_list = clean_doc(text)
print(tokens_list[:200])
print('Total Tokens: %d' % len(tokens_list))
print('Unique Tokens: %d' % len(set(tokens_list)))

['look', 'i', 'was', 'gonna', 'go', 'easy', 'on', 'you', 'not', 'to', 'hurt', 'your', 'feelings', 'but', 'im', 'only', 'going', 'to', 'get', 'this', 'one', 'chance', 'somethings', 'wrong', 'i', 'can', 'feel', 'it', 'just', 'a', 'feeling', 'ive', 'got', 'like', 'somethings', 'about', 'to', 'happen', 'but', 'i', 'dont', 'know', 'what', 'if', 'that', 'means', 'what', 'i', 'think', 'it', 'means', 'were', 'in', 'trouble', 'big', 'trouble', 'and', 'if', 'he', 'is', 'as', 'bananas', 'as', 'you', 'say', 'im', 'not', 'taking', 'any', 'chances', 'you', 'are', 'just', 'what', 'the', 'doc', 'ordered', 'im', 'beginnin', 'to', 'feel', 'like', 'a', 'rap', 'god', 'rap', 'god', 'all', 'my', 'people', 'from', 'the', 'front', 'to', 'the', 'back', 'nod', 'back', 'nod', 'now', 'who', 'thinks', 'their', 'arms', 'are', 'long', 'enough', 'to', 'slap', 'box', 'slap', 'box', 'they', 'said', 'i', 'rap', 'like', 'a', 'robot', 'so', 'call', 'me', 'rapbot', 'but', 'for', 'me', 'to', 'rap', 'like', 'a', 'computer', 

In [12]:
# Organizing tokens into sequences
seq_length = 50 + 1
sequences = list()
for i in range(seq_length, len(tokens_list)):
	# select sequence of tokens
	seq = tokens_list[i-seq_length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 224058


In [0]:
# save sequences to file
out_filename = 'final_eminem_lyrics.txt'
save_doc(sequences, out_filename)

In [0]:
# load doc into memory
def load_doc(filepath):
	# open the file as read only
	file = open(filepath, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = 'final_eminem_lyrics.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [0]:
# Using Keras tokenizer to tokenizer and sequence data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
len(tokenizer.word_index)
vocab_size = len(tokenizer.word_index) + 1

In [0]:
# separate into input and output
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [17]:
# Designing LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=seq_length)) # 300 is dimension size
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 300)           4632600   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           160400    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 15442)             1559642   
Total params: 6,443,142
Trainable params: 6,443,142
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Finally... FIT THIS MODEL!

In [0]:
# Fit model
model.fit(X, y, batch_size=128, epochs=30)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/30





210176/224058 [===========================>..] - ETA: 34s - loss: 6.7082 - acc: 0.0380

In [6]:
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

NameError: ignored

Re-Loading Data and using Trained Model to Generate Text

In [2]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

# load cleaned text sequences
in_filename = 'final_eminem_lyrics.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('model.h5')

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

FileNotFoundError: ignored